In [138]:
import csv
import pandas as pd
import re

In [139]:
sw = pd.read_csv('isw.csv')
sw

,col0,col1,col2,col3,col4
0,24-02-2022,Russia-Ukraine Warning Update: Initial Russian...,https://www.understandingwar.org/backgrounder/...,"<!DOCTYPE html>\r\n\r\n<html dir=""ltr"" lang=""e...","\r\nMason Clark, George Barros, and Kateryna S..."
1,25-02-2022,Russia-Ukraine Warning Update: Russian Offensi...,https://www.understandingwar.org/backgrounder/...,"<!DOCTYPE html>\r\n\r\n<html dir=""ltr"" lang=""e...",Key Takeaways\r\nRussian forces carried out ad...
2,26-02-2022,Russia-Ukraine Warning Update: Russian Offensi...,https://www.understandingwar.org/backgrounder/...,"<!DOCTYPE html>\r\n\r\n<html dir=""ltr"" lang=""e...",Ukrainian resistance remains remarkably effect...
3,27-02-2022,Russia-Ukraine Warning Update: Russian Offensi...,https://www.understandingwar.org/backgrounder/...,"<!DOCTYPE html>\r\n\r\n<html dir=""ltr"" lang=""e...",\r\nThe Russian military has likely recognized...
4,28-02-2022,"Russian Offensive Campaign Assessment, Februar...",https://www.understandingwar.org/backgrounder/...,"<!DOCTYPE html>\r\n\r\n<html dir=""ltr"" lang=""e...",The Russian military is reorganizing its milit...
...,...,...,...,...,...
331,21-01-2023,"Russian Offensive Campaign Assessment, January...",https://www.understandingwar.org/backgrounder/...,"<!DOCTYPE html>\r\n\r\n<html dir=""ltr"" lang=""e...",The Ukrainian defense of Bakhmut is likely a s...
332,22-01-2023,"Russian Offensive Campaign Assessment, Januar...",https://www.understandingwar.org/backgrounder/...,"<!DOCTYPE html>\r\n\r\n<html dir=""ltr"" lang=""e...","January 22, 8:30 pm ET\r\nISW is publishing an..."
333,23-01-2023,"Russian Offensive Campaign Assessment, January...",https://www.understandingwar.org/backgrounder/...,"<!DOCTYPE html>\r\n\r\n<html dir=""ltr"" lang=""e...","January 23, 9:45pm ET \r\nUkrainian intelligen..."
334,24-01-2023,"Russian Offensive Campaign Assessment, January...",https://www.understandingwar.org/backgrounder/...,"<!DOCTYPE html>\r\n\r\n<html dir=""ltr"" lang=""e...",A coalition of NATO member states reportedly w...


In [140]:
sw.rename(columns={'col0': 'date', 'col1': 'title', 'col2': 'url', 'col3': 'html', 'col4': 'main_text'}, inplace=True)

In [141]:
sw.tail(3)

,date,title,url,html,main_text
333,23-01-2023,"Russian Offensive Campaign Assessment, January...",https://www.understandingwar.org/backgrounder/...,"<!DOCTYPE html>\r\n\r\n<html dir=""ltr"" lang=""e...","January 23, 9:45pm ET \r\nUkrainian intelligen..."
334,24-01-2023,"Russian Offensive Campaign Assessment, January...",https://www.understandingwar.org/backgrounder/...,"<!DOCTYPE html>\r\n\r\n<html dir=""ltr"" lang=""e...",A coalition of NATO member states reportedly w...
335,25-01-2023,"Russian Offensive Campaign Assessment, January...",https://www.understandingwar.org/backgrounder/...,"<!DOCTYPE html>\r\n\r\n<html dir=""ltr"" lang=""e...",Russian forces may be engaging in limited spo...


In [142]:
sw[sw["main_text"].isna()]

,date,title,url,html,main_text
273,24-11-2022,NaN,NaN,NaN,NaN
304,25-12-2022,NaN,NaN,NaN,NaN
311,01-01-2023,NaN,NaN,NaN,NaN


In [143]:
#convert all the data in the column "main_text" to str
sw['main_text'] = sw['main_text'].astype(str)

In [144]:
sw.dtypes

date         object
title        object
url          object
html         object
main_text    object
dtype: object

In [145]:
#sw[sw["date"].str.endswith("08-2022")]

In [146]:
#remove authors
texts_auth = [
    "\r\nMason Clark, George Barros, and Kateryna Stepanenko\r\nFebruary 24, 3:00 pm EST\r\n",
    "Karolina Hird, Kateryna Stepanenko, and Mason Clark\r\nJune 9, 6:45 pm ET\r\n",
    "Frederick W. Kagan\r\nSeptember 25, 6 pm ET\r\n ",
    "Kateryna Stepanenko and Frederick W. Kagan\r\nOctober 9, 9:35 pm ET\r\n",
    "\r\n\r\nKarolina Hird, Grace Mappes, George Barros, Katherine Lawlor, and Mason Clark \r\nJanuary 10, 8:00 pm ET\r\n "
]

for text in texts_auth:
    sw['main_text'] = sw['main_text'].str.replace(text, '')

In [147]:
#remove the dates
date_pattern = r'\b(?:January|February|March|April|May|June|July|August|September|October|November|December)\s+\d{1,2},\s+(?:\d{1,2}:\d{2}\s*(?:am|pm)?\s*ET?|\d+(?:\s*pm|am)?\s*ET?)\b'
sw['main_text'] = sw['main_text'].apply(lambda x: re.sub(date_pattern, '', x).lstrip())

specific_pattern = r'\bDecember 8, 6:40 PM ET\b'
sw['main_text'] = sw['main_text'].apply(lambda x: re.sub(specific_pattern, '', x).lstrip())

In [148]:
#remove symbols [1], [2], ... [n]
unnecessary_numeration = r'\[\d+\]'
sw['main_text'] = sw['main_text'].apply(lambda x: re.sub(unnecessary_numeration, '', x))

In [149]:
#remove symbols "\r\n" and "\xa0"
sw['main_text'] = sw['main_text'].replace({'\r\n': ' ', '\xa0': ' ', '\xad': ''}, regex=True)

In [150]:
#unnecessary text in rows
un_pattern = r'We do not report in detail on Russian war crimes because those activities are well-covered in Western media and do not directly affect the military operations we are assessing and forecasting. We will continue to evaluate and report on the effects of these criminal activities on the Ukrainian military and population and specifically on combat in Ukrainian urban areas. We utterly condemn these Russian violations of the laws of armed conflict, Geneva Conventions, and humanity even though we do not describe them in these reports.'
un_cases = sw[sw['main_text'].str.contains(un_pattern, regex=True)]
un_cases

,date,title,url,html,main_text
41,06-04-2022,"Russian Offensive Campaign Assessment, April 6...",https://www.understandingwar.org/backgrounder/...,"<!DOCTYPE html>\r\n\r\n<html dir=""ltr"" lang=""e...",Russian forces continued to redeploy forces to...
42,07-04-2022,"Russian Offensive Campaign Assessment, April 7...",https://www.understandingwar.org/backgrounder/...,"<!DOCTYPE html>\r\n\r\n<html dir=""ltr"" lang=""e...",Russian proxy forces claimed to have captured ...
43,08-04-2022,"Russian Offensive Campaign Assessment, April 8...",https://www.understandingwar.org/backgrounder/...,"<!DOCTYPE html>\r\n\r\n<html dir=""ltr"" lang=""e...",Ukrainian forces retain control of defensive p...
44,09-04-2022,"Russian Offensive Campaign Assessment, April 9...",https://www.understandingwar.org/backgrounder/...,"<!DOCTYPE html>\r\n\r\n<html dir=""ltr"" lang=""e...",Special Edition: Russian Military Capabilities...
45,10-04-2022,"Russian Offensive Campaign Assessment, April 1...",https://www.understandingwar.org/backgrounder/...,"<!DOCTYPE html>\r\n\r\n<html dir=""ltr"" lang=""e...",Russian forces made territorial gains in Mariu...
...,...,...,...,...,...
330,20-01-2023,"Russian Offensive Campaign Assessment, January...",https://www.understandingwar.org/backgrounder/...,"<!DOCTYPE html>\r\n\r\n<html dir=""ltr"" lang=""e...",Head of the Chechen Republic Ramzan Kadyrov co...
331,21-01-2023,"Russian Offensive Campaign Assessment, January...",https://www.understandingwar.org/backgrounder/...,"<!DOCTYPE html>\r\n\r\n<html dir=""ltr"" lang=""e...",The Ukrainian defense of Bakhmut is likely a s...
333,23-01-2023,"Russian Offensive Campaign Assessment, January...",https://www.understandingwar.org/backgrounder/...,"<!DOCTYPE html>\r\n\r\n<html dir=""ltr"" lang=""e...",Ukrainian intelligence assessed that Russian f...
334,24-01-2023,"Russian Offensive Campaign Assessment, January...",https://www.understandingwar.org/backgrounder/...,"<!DOCTYPE html>\r\n\r\n<html dir=""ltr"" lang=""e...",A coalition of NATO member states reportedly w...


In [151]:
#remove unnecessary text in rows
unwanted_text = "We do not report in detail on Russian war crimes because those activities are well-covered in Western media and do not directly affect the military operations we are assessing and forecasting. We will continue to evaluate and report on the effects of these criminal activities on the Ukrainian military and population and specifically on combat in Ukrainian urban areas. We utterly condemn these Russian violations of the laws of armed conflict, Geneva Conventions, and humanity even though we do not describe them in these reports."

sw['main_text'] = sw['main_text'].replace(unwanted_text, '', regex=True)

In [152]:
#checkup
un_pattern = r'We do not report in detail on Russian war crimes because those activities are well-covered in Western media and do not directly affect the military operations we are assessing and forecasting. We will continue to evaluate and report on the effects of these criminal activities on the Ukrainian military and population and specifically on combat in Ukrainian urban areas. We utterly condemn these Russian violations of the laws of armed conflict, Geneva Conventions, and humanity even though we do not describe them in these reports.'
un_cases = sw[sw['main_text'].str.contains(un_pattern, regex=True)]
un_cases

,date,title,url,html,main_text


In [153]:
#remove "Key Takeaways" text
key_text = "Key Takeaways"
sw['main_text'] = sw['main_text'].replace(key_text, '', regex=True)

In [154]:
sw.tail(5)

,date,title,url,html,main_text
331,21-01-2023,"Russian Offensive Campaign Assessment, January...",https://www.understandingwar.org/backgrounder/...,"<!DOCTYPE html>\r\n\r\n<html dir=""ltr"" lang=""e...",The Ukrainian defense of Bakhmut is likely a s...
332,22-01-2023,"Russian Offensive Campaign Assessment, Januar...",https://www.understandingwar.org/backgrounder/...,"<!DOCTYPE html>\r\n\r\n<html dir=""ltr"" lang=""e...",ISW is publishing an abbreviated campaign upda...
333,23-01-2023,"Russian Offensive Campaign Assessment, January...",https://www.understandingwar.org/backgrounder/...,"<!DOCTYPE html>\r\n\r\n<html dir=""ltr"" lang=""e...",Ukrainian intelligence assessed that Russian f...
334,24-01-2023,"Russian Offensive Campaign Assessment, January...",https://www.understandingwar.org/backgrounder/...,"<!DOCTYPE html>\r\n\r\n<html dir=""ltr"" lang=""e...",A coalition of NATO member states reportedly w...
335,25-01-2023,"Russian Offensive Campaign Assessment, January...",https://www.understandingwar.org/backgrounder/...,"<!DOCTYPE html>\r\n\r\n<html dir=""ltr"" lang=""e...",Russian forces may be engaging in limited spoi...


In [155]:
#remove unnecessary text in rows
imm_text = "Immediate items to watch"
sw['main_text'] = sw['main_text'].replace(imm_text, '', regex=True)

In [156]:
#remove unnecessary text in rows
unw_text = r'ISW is publishing an abbreviated campaign update today, (January|October|November|December) \d{1,2}\.'

sw['main_text'] = sw['main_text'].apply(lambda x: re.sub(unw_text, '', x))

In [157]:
#remove unnecessary text in rows
unwn_text = r'Note: ISW does not receive any classified material from any source, uses only publicly available information, and draws extensively on Russian, Ukrainian, and Western reporting and social media as well as commercially available satellite imagery and other geospatial data as the basis for these reports. References to all sources used are provided in the endnotes of each update.'

sw['main_text'] = sw['main_text'].replace(unwn_text, '', regex=True)

In [158]:
u1_text = "Note: ISW and CTP will not publish a campaign assessment (or maps) tomorrow, December 25, in observance of the Christmas holiday. Coverage will resume Monday, December 26."
u2_text = "Note: ISW and CTP will not publish a campaign assessment (or maps) tomorrow, January 1, in observance of the New Year\'s Holiday. Coverage will resume on Monday, January 2."

sw['main_text'] = sw['main_text'].str.replace(u1_text, '', regex=False)
sw['main_text'] = sw['main_text'].str.replace(u2_text, '', regex=False)

In [159]:
#remove unnecessary text in rows
unss_text = r'This new section in the daily update is not in itself a forecast or assessment. It lays out the daily observed indicators we are using to refine our assessments and forecasts, which we expect to update regularly. Our assessment that the MDCOA remains unlikely has not changed.'
sw['main_text'] = sw['main_text'].replace(unss_text, '', regex=True)

unss_text = r'We will update this header if and when the assessment changes.'
sw['main_text'] = sw['main_text'].replace(unss_text, '', regex=True)

unss_text = r'Observed indicators for the MDCOA in the past 24 hours:'
sw['main_text'] = sw['main_text'].replace(unss_text, '', regex=True)

unss_text = r'Observed ambiguous indicators for MDCOA in the past 24 hours:'
sw['main_text'] = sw['main_text'].replace(unss_text, '', regex=True)

unss_text = r'Observed counter-indicators for the MDCOA in the past 24 hours:'
sw['main_text'] = sw['main_text'].replace(unss_text, '', regex=True)

unss_text = r'We will update this header if the assessment changes.'
sw['main_text'] = sw['main_text'].str.replace(unss_text, '', regex=False)

In [160]:
#remove quotation marks
sw['main_text'] = sw['main_text'].apply(lambda x: re.sub(r'[“”]', '', x))

sw['main_text'] = sw['main_text'].apply(lambda x: re.sub(r'"', '', x))

In [161]:
un_text = 'ISW does not receive any classified material from any source, uses only publicly available information, and draws extensively on Russian, Ukrainian, and Western reporting and social media as well as commercially available satellite imagery and other geospatial data as the basis for these reports.'
sw['main_text'] = sw['main_text'].str.replace(un_text, '', regex=False)


un_text = 'ISW specifically does not receive information from Prigozhin’s deceased mother-in-law, as he (ironically) suggested.'
sw['main_text'] = sw['main_text'].str.replace(un_text, '', regex=False)

un_text = 'Note: ISW will report on activities in Kherson Oblast as part of the Southern Axis in this and subsequent updates.'
sw['main_text'] = sw['main_text'].str.replace(un_text, '', regex=False)

In [173]:
unn_pattern = r'"'

unn_cases = sw[sw['main_text'].str.contains(unn_pattern, regex=True)]

unn_cases

,date,title,url,html,main_text


In [174]:
#remove extra spaces
sw['main_text'] = sw['main_text'].apply(lambda x: re.sub(r'\s+', ' ', x.strip()).lower())

In [175]:
sw.loc[303, 'main_text']

"russian forces’ rate of advance in the bakhmut area has likely slowed in recent days, although it is too early to assess whether the russian offensive to capture bakhmut has culminated. russian milbloggers acknowledged that ukrainian forces in the bakhmut area have managed to slightly slow down the pace of the russian advance around bakhmut and its surrounding settlements, with one claiming that ukrainian forces pushed back elements of the wagner group to positions they held days ago. ukrainian social media sources previously claimed that ukrainian forces completely pushed russian forces out of the eastern outskirts of bakhmut around december 21. isw has also assessed that russian forces made slightly fewer overall advances in the bakhmut area in november and december combined as compared to the month of october. russian forces will likely struggle to maintain the pace of their offensive operations in the bakhmut area and may seek to initiate a tactical or operational pause. the unite

In [176]:
sw.loc[199, 'main_text']

'ukrainian forces have inflicted a major operational defeat on russia, recapturing almost all kharkiv oblast in a rapid counter-offensive. the ukrainian success resulted from skillful campaign design and execution that included efforts to maximize the impact of western weapons systems such as himars. kyiv’s long discussion and then an announcement of a counter-offensive operation aimed at kherson oblast drew substantial russian troops away from the sectors on which ukrainian forces have conducted decisive attacks in the past several days. ukraine’s armed forces employed himars and other western systems to attack russian ground lines of communication (glocs) in kharkiv and kherson oblasts, setting conditions for the success of this operation. ukrainian leaders discussed the strikes in the south much more ostentatiously, however, successfully confusing the russians about their intentions in kharkiv oblast. western weapons systems were necessary but not sufficient to secure success for uk

In [177]:
sw.to_csv('updated_isw.csv', index=False)